# Anomaly Detection Models


- we are going to tyr isolation and dbscan.


## Models:
1. **Isolation Forest**: Tree-based anomaly detection
   - Fast, handles high dimensions, works well with numerical features
   - Requires contamination parameter tuning

2. **DBSCAN**: Density-based clustering
   - No k needed, finds clusters naturally, good for spatial patterns
   - Sensitive to eps/min_samples, slower on large datasets

## Evaluation:
- Since we have ground truth labels (`is_anomaly`), we can evaluate:
- Precision: Of detected anomalies, how many are real?
- Recall: Of real anomalies, how many did we detect?
- F1-Score: mean of precision and recall
- Confusion Matrix


- we are lookiing for finding more TP. so we care more about FN.
- because false detection is fine but. missing the anomaly is costly for us,
- so we care more about recall and at the same time it should not tag all the messages as anomaly.


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

## 1. Load and Merge All Features

In [2]:



global_features = pd.read_parquet('../data/features/global_features.parquet')
print(f"Global features: {global_features.shape}")

text_features = pd.read_parquet('../data/features/text_features.parquet')
print(f"Text features: {text_features.shape}")
print(f"  Global: {global_features.columns.tolist()[:10]}...")
print(f"  Text: {text_features.columns.tolist()[:10]}...")

Global features: (25921, 13)
Text features: (25921, 19)
  Global: ['timestamp', 'log_count', 'error_count', 'fatal_count', 'warn_count', 'info_count', 'debug_count', 'error_rate', 'fatal_rate', 'warn_rate']...
  Text: ['timestamp', 'message_rarity', 'message_anomaly_score', 'message_cluster', 'tfidf_0', 'tfidf_1', 'tfidf_2', 'tfidf_3', 'tfidf_4', 'tfidf_5']...


In [4]:
# Check timestamp formats and convert to timezone-naive.
for df, name in [(global_features, 'global'),
                  (text_features, 'text')]:
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)
    print(f"  {name}: {df['timestamp'].dtype}")

  global: datetime64[ns]
  text: datetime64[ns]


In [5]:
df_merged = global_features.copy()

df_merged = df_merged.merge(
    text_features,
    on='timestamp',
    how='inner',
    suffixes=('', '_text')
)
print(f"After text: {df_merged.shape}")
print(f"Total windows: {len(df_merged):,}")
print(f"Total features: {len(df_merged.columns)}")

df_merged.head()

After text: (25921, 31)
Total windows: 25,921
Total features: 31


,timestamp,log_count,error_count,fatal_count,warn_count,info_count,debug_count,error_rate,fatal_rate,warn_rate,...,tfidf_5,tfidf_6,tfidf_7,tfidf_8,tfidf_9,tfidf_10,tfidf_11,tfidf_12,tfidf_13,tfidf_14
0,2025-12-15 22:54:00,3,0,0,1,1,1,0.0,0.0,0.333333,...,-0.075225,0.004172,0.052505,-0.014076,0.185758,-0.019513,-0.026326,-4.196363e-11,-0.059728,0.000366
1,2025-12-15 22:54:30,5,0,0,1,2,2,0.0,0.0,0.200000,...,0.176190,-0.004748,0.149032,-0.030070,0.019358,-0.014833,-0.001962,-9.281976e-10,-0.005273,0.129338
2,2025-12-15 22:55:00,6,0,0,0,4,2,0.0,0.0,0.000000,...,0.015996,0.024430,-0.121851,-0.119422,0.002459,-0.011482,0.002484,3.333333e-01,-0.000151,-0.066615
3,2025-12-15 22:55:30,5,0,0,2,2,1,0.0,0.0,0.400000,...,-0.077468,-0.029707,-0.068632,0.015042,0.139515,0.188312,-0.041648,-1.941217e-11,-0.050769,0.001923
4,2025-12-15 22:56:00,5,0,0,2,2,1,0.0,0.0,0.400000,...,-0.054243,-0.028855,0.168491,-0.056872,0.165562,0.124260,-0.051269,-9.018995e-11,0.131728,-0.012412


## 2. Load Ground Truth Labels

In [6]:
# Load original training data to get ground truth labels per window.
logs = pd.read_parquet('../data/training_logs.parquet')
logs['timestamp'] = pd.to_datetime(logs['timestamp']).dt.tz_localize(None)

print(f"Total logs: {len(logs):,}")
print(f"Anomalies: {logs['is_anomaly'].sum():,} ({logs['is_anomaly'].mean()*100:.2f}%)")

# Aggregate to 30s windows - mark window as anomaly if ANY log in window is anomaly.
window_labels = logs.set_index('timestamp').resample('30s').agg({
    'is_anomaly': 'max'  # 1 if any anomaly in window, else 0.
}).reset_index()

print(f"Total windows: {len(window_labels):,}")
print(f"Anomaly windows: {window_labels['is_anomaly'].sum():,} ({window_labels['is_anomaly'].mean()*100:.2f}%)")

window_labels.head()

Total logs: 131,812
Anomalies: 758 (0.58%)
Total windows: 25,921
Anomaly windows: 104.0 (0.40%)


,timestamp,is_anomaly
0,2025-12-15 22:54:00,0.0
1,2025-12-15 22:54:30,0.0
2,2025-12-15 22:55:00,0.0
3,2025-12-15 22:55:30,0.0
4,2025-12-15 22:56:00,0.0


In [7]:
df_merged = df_merged.merge(
    window_labels,
    on='timestamp',
    how='inner'
)

print(f"  Shape: {df_merged.shape}")
print(f"  NaN values in is_anomaly: {df_merged['is_anomaly'].isna().sum()}")
df_merged['is_anomaly'] = df_merged['is_anomaly'].fillna(0).astype(int)

print(f"  Anomaly windows: {df_merged['is_anomaly'].sum():,} ({df_merged['is_anomaly'].mean()*100:.2f}%)")
y_true = df_merged['is_anomaly'].values

print(f"\nClass distribution:")
print(f"  Normal: {(y_true == 0).sum():,}")
print(f"  Anomaly: {(y_true == 1).sum():,}")

  Shape: (25921, 32)
  NaN values in is_anomaly: 150
  Anomaly windows: 104 (0.40%)

Class distribution:
  Normal: 25,817
  Anomaly: 104


## 3. Prepare Features for Training

In [8]:
exclude_cols = ['timestamp', 'is_anomaly', 'anomaly_type']
feature_cols = [col for col in df_merged.columns if col not in exclude_cols]

print(f"Total feature columns: {len(feature_cols)}")
print(f"\nFirst 20 features:")
print(feature_cols[:20])

X = df_merged[feature_cols].copy()
print(f"\nFeature matrix shap: {X.shape}")

Total feature columns: 30

First 20 features:
['log_count', 'error_count', 'fatal_count', 'warn_count', 'info_count', 'debug_count', 'error_rate', 'fatal_rate', 'warn_rate', 'warn_to_info_ratio', 'critical_count', 'critical_rate', 'message_rarity', 'message_anomaly_score', 'message_cluster', 'tfidf_0', 'tfidf_1', 'tfidf_2', 'tfidf_3', 'tfidf_4']

Feature matrix shap: (25921, 30)


In [9]:
# Check for missing values and infinities.
print("Checking data quality...")
print(f"  Missing values: {X.isnull().sum().sum():,}")
print(f"  Infinite values: {np.isinf(X.select_dtypes(include=[np.number])).sum().sum():,}")

if X.isnull().sum().sum() > 0:
    print("\n  Filling missing values with 0...")
    X = X.fillna(0)

if np.isinf(X.select_dtypes(include=[np.number])).sum().sum() > 0:
    print("  Replacing infinite values with 0...")
    X = X.replace([np.inf, -np.inf], 0)


Checking data quality...
  Missing values: 2,550
  Infinite values: 0

  Filling missing values with 0...


In [10]:
# Standardize features-----> important for DBSCAN since this is affected by distance.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"  Mean: {X_scaled.mean():.6f}")
print(f"  Std: {X_scaled.std():.6f}")
print(f"  Shape: {X_scaled.shape}")

  Mean: 0.000000
  Std: 1.000000
  Shape: (25921, 30)


## 4. Model 1: Isolation Forest

In [ ]:

# Define parameter grid.
# Contamination: proportion of anomalies (test around actual rate of 0.4%).
contamination_values = [0.001, 0.002, 0.004, 0.006, 0.008, 0.01, 0.015, 0.02, 0.03, 0.05]
n_estimators_values = [50, 100, 150, 200]
iso_tuning_results = []

print(f"Testing {len(contamination_values)} contamination values × {len(n_estimators_values)} n_estimators values = {len(contamination_values) * len(n_estimators_values)} combinations\n")
for contamination in contamination_values:
    for n_estimators in n_estimators_values:
        iso_temp = IsolationForest(
            n_estimators=n_estimators,
            contamination=contamination,
            random_state=42,
            n_jobs=-1,
            verbose=0
        )
        iso_temp.fit(X_scaled)
        predictions = iso_temp.predict(X_scaled)
        y_pred = (predictions == -1).astype(int)
        n_predicted = y_pred.sum()
        
        if n_predicted == 0 or n_predicted == len(y_pred):
            precision, recall, f1 = 0.0, 0.0, 0.0
        else:
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_true, y_pred, average='binary', zero_division=0
            )
        
        iso_tuning_results.append({
            'contamination': contamination,
            'n_estimators': n_estimators,
            'n_predicted': n_predicted,
            'predicted_pct': n_predicted / len(y_pred) * 100,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        })
        
        print(f"contamination={contamination:.4f}, n_estimators={n_estimators:3d} → predicted={n_predicted:5d} ({n_predicted/len(y_pred)*100:5.2f}%) → P={precision:.3f}, R={recall:.3f}, F1={f1:.3f}")



Isolation Forest Hyperparameter Tuning
Testing 10 contamination values × 4 n_estimators values = 40 combinations

contamination=0.0010, n_estimators= 50 → predicted=   26 ( 0.10%) → P=1.000, R=0.250, F1=0.400
contamination=0.0010, n_estimators=100 → predicted=   26 ( 0.10%) → P=1.000, R=0.250, F1=0.400
contamination=0.0010, n_estimators=150 → predicted=   26 ( 0.10%) → P=1.000, R=0.250, F1=0.400
contamination=0.0010, n_estimators=200 → predicted=   26 ( 0.10%) → P=1.000, R=0.250, F1=0.400
contamination=0.0020, n_estimators= 50 → predicted=   52 ( 0.20%) → P=0.808, R=0.404, F1=0.538
contamination=0.0020, n_estimators=100 → predicted=   52 ( 0.20%) → P=0.865, R=0.433, F1=0.577
contamination=0.0020, n_estimators=150 → predicted=   52 ( 0.20%) → P=0.808, R=0.404, F1=0.538
contamination=0.0020, n_estimators=200 → predicted=   52 ( 0.20%) → P=0.788, R=0.394, F1=0.526
contamination=0.0040, n_estimators= 50 → predicted=  104 ( 0.40%) → P=0.529, R=0.529, F1=0.529
contamination=0.0040, n_estimat

In [92]:
results_df = pd.DataFrame(iso_tuning_results)
best_idx = results_df['f1_score'].idxmax()
best_params = results_df.loc[best_idx]
best_params

contamination      0.002000
n_estimators     100.000000
n_predicted       52.000000
predicted_pct      0.200610
precision          0.865385
recall             0.432692
f1_score           0.576923
Name: 5, dtype: float64

In [11]:
print("Training Isolation Forest...")
print("="*60)

# this Tells the model "approximately X% of my data points are anomalies"
# Used to determines the threshold for classifying points as anomalies

contamination = y_true.mean()

# check contamination is valid (between 0 and 0.5).
if np.isnan(contamination) or contamination == 0 or contamination > 0.5:
    print(f"Invalid contamination={contamination}, using default 0.01")
    contamination = 0.01

print(f"Contamination parameter: {contamination:.4f}")

iso_forest = IsolationForest(
    n_estimators=100,
    contamination=contamination,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

iso_forest.fit(X_scaled)
print(f"  Number of trees: {iso_forest.n_estimators}")
print(f"  Max samples: {iso_forest.max_samples}")

Training Isolation Forest...
Contamination parameter: 0.0040
  Number of trees: 100
  Max samples: auto


In [12]:
# Returns: 1 for noraml, -1 for outliers/anomalies.
iso_predictions = iso_forest.predict(X_scaled)
y_pred_iso = (iso_predictions == -1).astype(int) # Convert to binary: 0 = normal, 1 = anomaly.
# Get anomaly scores (lower = more anomalous).
iso_scores = iso_forest.score_samples(X_scaled)

print(f"  Predicted anomalies: {y_pred_iso.sum():,} ({y_pred_iso.mean()*100:.2f}%)")
print(f"  Anomaly score range: [{iso_scores.min():.4f}, {iso_scores.max():.4f}]")

  Predicted anomalies: 104 (0.40%)
  Anomaly score range: [-0.7026, -0.3584]


In [13]:
print("\nIsolation Forest Performance:")
print("="*60)
print(classification_report(
    y_true,
    y_pred_iso,
    target_names=['Normal', 'Anomaly'],
    digits=4
))

cm_iso = confusion_matrix(y_true, y_pred_iso)
print(f"  True Negatives:  {cm_iso[0, 0]:6,}")
print(f"  False Positives: {cm_iso[0, 1]:6,}")
print(f"  False Negatives: {cm_iso[1, 0]:6,}")
print(f"  True Positives:  {cm_iso[1, 1]:6,}")


Isolation Forest Performance:
              precision    recall  f1-score   support

      Normal     0.9979    0.9979    0.9979     25817
     Anomaly     0.4904    0.4904    0.4904       104

    accuracy                         0.9959     25921
   macro avg     0.7442    0.7442    0.7442     25921
weighted avg     0.9959    0.9959    0.9959     25921

  True Negatives:  25,764
  False Positives:     53
  False Negatives:     53
  True Positives:      51


## 5. Model 2: DBSCAN

### 5.1 DBSCAN parameter tuning

In [19]:
eps_values = [2.0, 3.0, 5.0, 7.0, 10.0, 15.0, 20.0]
min_samples_values = [3, 5, 10, 15, 20]
tuning_results = []

print(f"Testing {len(eps_values)} eps values × {len(min_samples_values)} min_samples values = {len(eps_values) * len(min_samples_values)} combinations\n")

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan_temp = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
        labels = dbscan_temp.fit_predict(X_scaled)
        
        y_pred = (labels == -1).astype(int)
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        # Skip if no anomalies detected or all points are anomalies.
        if y_pred.sum() == 0 or y_pred.sum() == len(y_pred):
            precision, recall, f1 = 0.0, 0.0, 0.0
        else:
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_true, y_pred, average='binary', zero_division=0
            )
        
        tuning_results.append({
            'eps': eps,
            'min_samples': min_samples,
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'noise_pct': n_noise / len(y_pred) * 100,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        })
        
        print(f"eps={eps:5.1f}, min_samples={min_samples:2d} → clusters={n_clusters:2d}, noise={n_noise:5d} ({n_noise/len(y_pred)*100:5.2f}%) → P={precision:.3f}, R={recall:.3f}, F1={f1:.3f}")



Testing 7 eps values × 5 min_samples values = 35 combinations

eps=  2.0, min_samples= 3 → clusters=998, noise=11004 (42.45%) → P=0.009, R=1.000, F1=0.019
eps=  2.0, min_samples= 5 → clusters=433, noise=14844 (57.27%) → P=0.007, R=1.000, F1=0.014
eps=  2.0, min_samples=10 → clusters=153, noise=20302 (78.32%) → P=0.005, R=1.000, F1=0.010
eps=  2.0, min_samples=15 → clusters=63, noise=23030 (88.85%) → P=0.005, R=1.000, F1=0.009
eps=  2.0, min_samples=20 → clusters=27, noise=24220 (93.44%) → P=0.004, R=1.000, F1=0.009
eps=  3.0, min_samples= 3 → clusters=131, noise=  930 ( 3.59%) → P=0.110, R=0.981, F1=0.197
eps=  3.0, min_samples= 5 → clusters=87, noise= 1405 ( 5.42%) → P=0.073, R=0.990, F1=0.137
eps=  3.0, min_samples=10 → clusters=40, noise= 2473 ( 9.54%) → P=0.042, R=0.990, F1=0.080
eps=  3.0, min_samples=15 → clusters=30, noise= 3403 (13.13%) → P=0.030, R=0.990, F1=0.059
eps=  3.0, min_samples=20 → clusters=19, noise= 4256 (16.42%) → P=0.024, R=0.990, F1=0.047
eps=  5.0, min_samples=

In [20]:
results_df = pd.DataFrame(tuning_results)
best_idx = results_df['f1_score'].idxmax()
best_params = results_df.loc[best_idx]

print("\n" + "="*60)
print("BEST PARAMETERS")
print("="*60)
print(f"  eps: {best_params['eps']}")
print(f"  min_samples: {best_params['min_samples']}")
print(f"  F1-Score: {best_params['f1_score']:.4f}")
print(f"  Precision: {best_params['precision']:.4f}")
print(f"  Recall: {best_params['recall']:.4f}")
print(f"  Clusters: {int(best_params['n_clusters'])}")
print(f"  Noise points: {int(best_params['n_noise'])} ({best_params['noise_pct']:.2f}%)")

# Show top 5 configurations.
print("\n" + "="*60)
print("TOP 5 CONFIGURATIONS BY F1-SCORE")
print("="*60)
top5 = results_df.nlargest(5, 'f1_score')[['eps', 'min_samples', 'precision', 'recall', 'f1_score', 'n_clusters', 'noise_pct']]
print(top5.to_string(index=False))


BEST PARAMETERS
  eps: 7.0
  min_samples: 15.0
  F1-Score: 0.7273
  Precision: 0.8889
  Recall: 0.6154
  Clusters: 1
  Noise points: 72 (0.28%)

TOP 5 CONFIGURATIONS BY F1-SCORE
 eps  min_samples  precision   recall  f1_score  n_clusters  noise_pct
 7.0           15   0.888889 0.615385  0.727273           1   0.277767
 7.0           20   0.853333 0.615385  0.715084           1   0.289341
 5.0            5   0.729167 0.673077  0.700000           7   0.370356
 5.0           10   0.647541 0.759615  0.699115           4   0.470661
 7.0           10   0.882353 0.576923  0.697674           1   0.262336


In [21]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'F1-Score Heatmap',
        'Precision Heatmap',
        'Recall Heatmap',
        'Number of Noise Points'
    ),
    specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}],
           [{'type': 'heatmap'}, {'type': 'heatmap'}]]
)

# Create pivot tables for heatmaps.
f1_pivot = results_df.pivot(index='min_samples', columns='eps', values='f1_score')
precision_pivot = results_df.pivot(index='min_samples', columns='eps', values='precision')
recall_pivot = results_df.pivot(index='min_samples', columns='eps', values='recall')
noise_pivot = results_df.pivot(index='min_samples', columns='eps', values='noise_pct')

# F1-Score heatmap.
fig.add_trace(
    go.Heatmap(
        z=f1_pivot.values,
        x=f1_pivot.columns,
        y=f1_pivot.index,
        colorscale='Viridis',
        text=np.round(f1_pivot.values, 3),
        texttemplate='%{text}',
        colorbar=dict(x=0.46)
    ),
    row=1, col=1
)

# Precision heatmap.
fig.add_trace(
    go.Heatmap(
        z=precision_pivot.values,
        x=precision_pivot.columns,
        y=precision_pivot.index,
        colorscale='Blues',
        text=np.round(precision_pivot.values, 3),
        texttemplate='%{text}',
        colorbar=dict(x=1.02)
    ),
    row=1, col=2
)

# Recall heatmap.
fig.add_trace(
    go.Heatmap(
        z=recall_pivot.values,
        x=recall_pivot.columns,
        y=recall_pivot.index,
        colorscale='Reds',
        text=np.round(recall_pivot.values, 3),
        texttemplate='%{text}',
        colorbar=dict(x=0.46)
    ),
    row=2, col=1
)

# Noise percentage heatmap.
fig.add_trace(
    go.Heatmap(
        z=noise_pivot.values,
        x=noise_pivot.columns,
        y=noise_pivot.index,
        colorscale='Oranges',
        text=np.round(noise_pivot.values, 2),
        texttemplate='%{text}%',
        colorbar=dict(x=1.02)
    ),
    row=2, col=2
)

# Update axes.
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(title_text='eps', row=i, col=j)
        fig.update_yaxes(title_text='min_samples', row=i, col=j)

fig.update_layout(
    title='DBSCAN Hyperparameter Tuning Results',
    height=800,
    showlegend=False
)
fig.show()


### 5.2 Train DBSCAN with Best Parameters


In [ ]:
best_eps = best_params['eps']
best_min_samples = int(best_params['min_samples'])

print(f"  eps: {best_eps}")
print(f"  min_samples: {best_min_samples}")

dbscan = DBSCAN(
    eps=7,
    min_samples=15,
    n_jobs=-1
)

cluster_labels = dbscan.fit_predict(X_scaled)



# best param:
#  eps: 7.0
#   min_samples: 15

  eps: 7.0
  min_samples: 15


In [23]:
# DBSCAN assigns cluster IDs to normal points, -1 to noise/anomalies.
n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
n_noise = list(cluster_labels).count(-1)

print(f"Clustering results:")
print(f"  Number of clusters: {n_clusters}")
print(f"  Noise points (anomalies): {n_noise:,} ({n_noise/len(cluster_labels)*100:.2f}%)")

# Convert to binary: 0 = normal (in cluster), 1 = anomaly (noise).
y_pred_dbscan = (cluster_labels == -1).astype(int)
print(f"  Predicted anomalies: {y_pred_dbscan.sum():,} ({y_pred_dbscan.mean()*100:.2f}%)")

Clustering results:
  Number of clusters: 1
  Noise points (anomalies): 72 (0.28%)
  Predicted anomalies: 72 (0.28%)


In [24]:
print(classification_report(
    y_true,
    y_pred_dbscan,
    target_names=['Normal', 'Anomaly'],
    digits=4
))
cm_dbscan = confusion_matrix(y_true, y_pred_dbscan)
print("\nConfusion Matrix:")
print(f"  True Negatives:  {cm_dbscan[0, 0]:6,}")
print(f"  False Positives: {cm_dbscan[0, 1]:6,}")
print(f"  False Negatives: {cm_dbscan[1, 0]:6,}")
print(f"  True Positives:  {cm_dbscan[1, 1]:6,}")

              precision    recall  f1-score   support

      Normal     0.9985    0.9997    0.9991     25817
     Anomaly     0.8889    0.6154    0.7273       104

    accuracy                         0.9981     25921
   macro avg     0.9437    0.8075    0.8632     25921
weighted avg     0.9980    0.9981    0.9980     25921


Confusion Matrix:
  True Negatives:  25,809
  False Positives:      8
  False Negatives:     40
  True Positives:      64


## 6. Model Comparison

In [25]:
print("\nModel Comparison:")
print("="*60)

# Calculate metrics for both models.
iso_precision, iso_recall, iso_f1, _ = precision_recall_fscore_support(
    y_true, y_pred_iso, average='binary'
)

dbscan_precision, dbscan_recall, dbscan_f1, _ = precision_recall_fscore_support(
    y_true, y_pred_dbscan, average='binary'
)

comparison = pd.DataFrame({
    'Metric': ['Precision', 'Recall', 'F1-Score', 'Predicted Anomalies'],
    'Isolation Forest': [
        f"{iso_precision:.4f}",
        f"{iso_recall:.4f}",
        f"{iso_f1:.4f}",
        f"{y_pred_iso.sum():,}"
    ],
    'DBSCAN': [
        f"{dbscan_precision:.4f}",
        f"{dbscan_recall:.4f}",
        f"{dbscan_f1:.4f}",
        f"{y_pred_dbscan.sum():,}"
    ]
})

print(comparison.to_string(index=False))
print(f"\nTrue anomalies: {y_true.sum():,}")


Model Comparison:
             Metric Isolation Forest DBSCAN
          Precision           0.4904 0.8889
             Recall           0.4904 0.6154
           F1-Score           0.4904 0.7273
Predicted Anomalies              104     72

True anomalies: 104


In [26]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Isolation Forest', 'DBSCAN'),
    specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}]]
)

fig.add_trace(
    go.Heatmap(
        z=cm_iso,
        x=['Predicted Normal', 'Predicted Anomaly'],
        y=['True Normal', 'True Anomaly'],
        text=cm_iso,
        texttemplate='%{text}',
        colorscale='Blues',
        showscale=False
    ),
    row=1, col=1
)
fig.add_trace(
    go.Heatmap(
        z=cm_dbscan,
        x=['Predicted Normal', 'Predicted Anomaly'],
        y=['True Normal', 'True Anomaly'],
        text=cm_dbscan,
        texttemplate='%{text}',
        colorscale='Oranges',
        showscale=False
    ),
    row=1, col=2
)

fig.update_layout(
    title='Confusion Matrices Comparison',
    height=500
)
fig.show()

## 8. Save Best Model

In [27]:
dbscan_f1

0.7272727272727273

In [28]:
import joblib
import os

# Determine best model based on F1-score.
if iso_f1 > dbscan_f1:
    best_model = 'isolation_forest'
    best_f1 = iso_f1
    model_to_save = iso_forest
else:
    best_model = 'dbscan'
    best_f1 = dbscan_f1
    model_to_save = dbscan

print(f"\nBest Model: {best_model.upper()}")
print(f"  F1-Score: {best_f1:.4f}")

# Create models directory.
os.makedirs('../models', exist_ok=True)

# Save best model.
model_path = f'../models/{best_model}.pkl'
joblib.dump(model_to_save, model_path)
print(f"\n✓ Model saved: {model_path}")

# Save scaler (needed for inference).
scaler_path = '../models/scaler.pkl'
joblib.dump(scaler, scaler_path)
print(f"✓ Scaler saved: {scaler_path}")

# Save feature names.
feature_names_path = '../models/feature_names.txt'
with open(feature_names_path, 'w') as f:
    f.write('\n'.join(feature_cols))
print(f"✓ Feature names saved: {feature_names_path}")

# Save both models for comparison.
joblib.dump(iso_forest, '../models/isolation_forest.pkl')
joblib.dump(dbscan, '../models/dbscan.pkl')



Best Model: DBSCAN
  F1-Score: 0.7273

✓ Model saved: ../models/dbscan.pkl
✓ Scaler saved: ../models/scaler.pkl
✓ Feature names saved: ../models/feature_names.txt


['../models/dbscan.pkl']

## 10. Model 3: HDBSCAN

HDBSCAN (Hierarchical Density-Based Spatial Clustering) is an improvement over DBSCAN:
- **No eps parameter needed** - automatically finds optimal density threshold.
- **Handles varying density clusters** - better for complex patterns.
- **More robust** - less sensitive to parameter choices.
- **Hierarchical** - builds a cluster hierarchy for better anomaly detection.s

### 10.1 HDBSCAN Hyperparameter Tuning

Key parameters:
- **min_cluster_size**: Minimum size of clusters (smaller = more granular).
- **min_samples**: How conservative clustering should be (higher = fewer outliers).

In [29]:
import hdbscan
min_cluster_size_values = [5, 10, 15, 20, 30, 50, 100]
min_samples_values = [1, 3, 5, 10, 15, 20]
hdbscan_tuning_results = []

print(f"Testing {len(min_cluster_size_values)} min_cluster_size values × {len(min_samples_values)} min_samples values = {len(min_cluster_size_values) * len(min_samples_values)} combinations\n")

# Grid search.
for min_cluster_size in min_cluster_size_values:
    for min_samples in min_samples_values:
        hdbscan_temp = hdbscan.HDBSCAN(
            min_cluster_size=min_cluster_size,
            min_samples=min_samples,
            core_dist_n_jobs=-1
        )
        labels = hdbscan_temp.fit_predict(X_scaled)
        y_pred = (labels == -1).astype(int)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        if y_pred.sum() == 0 or y_pred.sum() == len(y_pred):
            precision, recall, f1 = 0.0, 0.0, 0.0
        else:
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_true, y_pred, average='binary', zero_division=0
            )
        
        hdbscan_tuning_results.append({
            'min_cluster_size': min_cluster_size,
            'min_samples': min_samples,
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'noise_pct': n_noise / len(y_pred) * 100,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        })
        
        print(f"min_cluster_size={min_cluster_size:3d}, min_samples={min_samples:2d} → clusters={n_clusters:2d}, noise={n_noise:5d} ({n_noise/len(y_pred)*100:5.2f}%) → P={precision:.3f}, R={recall:.3f}, F1={f1:.3f}")



Testing 7 min_cluster_size values × 6 min_samples values = 42 combinations

min_cluster_size=  5, min_samples= 1 → clusters=1250, noise= 9560 (36.88%) → P=0.006, R=0.519, F1=0.011
min_cluster_size=  5, min_samples= 3 → clusters=695, noise=14706 (56.73%) → P=0.005, R=0.731, F1=0.010
min_cluster_size=  5, min_samples= 5 → clusters=379, noise=16868 (65.07%) → P=0.005, R=0.837, F1=0.010
min_cluster_size=  5, min_samples=10 → clusters=128, noise=16244 (62.67%) → P=0.006, R=0.923, F1=0.012
min_cluster_size=  5, min_samples=15 → clusters=102, noise=18713 (72.19%) → P=0.005, R=0.942, F1=0.010
min_cluster_size=  5, min_samples=20 → clusters=90, noise=15320 (59.10%) → P=0.006, R=0.942, F1=0.013
min_cluster_size= 10, min_samples= 1 → clusters=380, noise=12378 (47.75%) → P=0.005, R=0.635, F1=0.011
min_cluster_size= 10, min_samples= 3 → clusters=191, noise=14801 (57.10%) → P=0.006, R=0.808, F1=0.011
min_cluster_size= 10, min_samples= 5 → clusters=161, noise=17356 (66.96%) → P=0.006, R=0.923, F1=0.0

In [30]:
hdbscan_results_df = pd.DataFrame(hdbscan_tuning_results)
hdbscan_best_idx = hdbscan_results_df['f1_score'].idxmax()
hdbscan_best_params = hdbscan_results_df.loc[hdbscan_best_idx]

print("\n" + "="*60)
print("BEST PARAMETERS")
print("="*60)
print(f"  min_cluster_size: {int(hdbscan_best_params['min_cluster_size'])}")
print(f"  min_samples: {int(hdbscan_best_params['min_samples'])}")
print(f"  F1-Score: {hdbscan_best_params['f1_score']:.4f}")
print(f"  Precision: {hdbscan_best_params['precision']:.4f}")
print(f"  Recall: {hdbscan_best_params['recall']:.4f}")
print(f"  Clusters: {int(hdbscan_best_params['n_clusters'])}")
print(f"  Noise points: {int(hdbscan_best_params['n_noise'])} ({hdbscan_best_params['noise_pct']:.2f}%)")

# Show top 5 configurations.
print("\n" + "="*60)
print("TOP 5 CONFIGURATIONS BY F1-SCORE")
print("="*60)
hdbscan_top5 = hdbscan_results_df.nlargest(5, 'f1_score')[['min_cluster_size', 'min_samples', 'precision', 'recall', 'f1_score', 'n_clusters', 'noise_pct']]
print(hdbscan_top5.to_string(index=False))



# ============================================================
# BEST PARAMETERS
# ============================================================
#   min_cluster_size: 10
#   min_samples: 1
#   F1-Score: 0.5877
#   Precision: 0.5403
#   Recall: 0.6442
#   Clusters: 10
#   Noise points: 124 (0.48%)

# ============================================================
# TOP 5 CONFIGURATIONS BY F1-SCORE
# ============================================================
#  min_cluster_size  min_samples  precision   recall  f1_score  n_clusters  noise_pct
#                10            1   0.540323 0.644231  0.587719          10   0.478377
#                10            3   0.432749 0.711538  0.538182          10   0.659697
#                 5            3   0.906977 0.375000  0.530612           5   0.165889
#                15            1   0.621622 0.442308  0.516854           2   0.285483
#                 5            5   0.393939 0.750000  0.516556          14   0.763859


BEST PARAMETERS
  min_cluster_size: 100
  min_samples: 1
  F1-Score: 0.5714
  Precision: 0.4545
  Recall: 0.7692
  Clusters: 3
  Noise points: 176 (0.68%)

TOP 5 CONFIGURATIONS BY F1-SCORE
 min_cluster_size  min_samples  precision   recall  f1_score  n_clusters  noise_pct
              100            1   0.454545 0.769231  0.571429           3   0.678986
              100            3   0.419512 0.826923  0.556634           3   0.790865
              100            5   0.370213 0.836538  0.513274           3   0.906601
              100           10   0.348000 0.836538  0.491525           3   0.964469
              100           15   0.268882 0.855769  0.409195           3   1.276957


### 10.2 Train HDBSCAN with Best Parameters

In [31]:
print("Training HDBSCAN with best parameters...")
print("="*60)
best_min_cluster_size = int(hdbscan_best_params['min_cluster_size'])
best_hdb_min_samples = int(hdbscan_best_params['min_samples'])

print(f"min_cluster_size: {best_min_cluster_size}")
print(f"min_samples: {best_hdb_min_samples}")

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=best_min_cluster_size,
    min_samples=best_hdb_min_samples,
    core_dist_n_jobs=-1
)
hdbscan_labels = hdbscan_model.fit_predict(X_scaled)


# HDBSCAN assigns cluster IDs to normal points, -1 to noise/anomalies.
hdb_n_clusters = len(set(hdbscan_labels)) - (1 if -1 in hdbscan_labels else 0)
hdb_n_noise = list(hdbscan_labels).count(-1)
print(f"  Number of clusters: {hdb_n_clusters}")
print(f"  Noise points (anomalies): {hdb_n_noise:,} ({hdb_n_noise/len(hdbscan_labels)*100:.2f}%)")
# Convert to binary: 0 = normal (in cluster), 1 = anomaly (noise).
y_pred_hdbscan = (hdbscan_labels == -1).astype(int)
print(f"  Predicted anomalies: {y_pred_hdbscan.sum():,} ({y_pred_hdbscan.mean()*100:.2f}%)")

Training HDBSCAN with best parameters...
min_cluster_size: 100
min_samples: 1
  Number of clusters: 3
  Noise points (anomalies): 176 (0.68%)
  Predicted anomalies: 176 (0.68%)


In [32]:

print(classification_report(
    y_true,
    y_pred_hdbscan,
    target_names=['Normal', 'Anomaly'],
    digits=4
))

cm_hdbscan = confusion_matrix(y_true, y_pred_hdbscan)
print("\nConfusion Matrix:")
print(f"  True Negatives:  {cm_hdbscan[0, 0]:6,}")
print(f"  False Positives: {cm_hdbscan[0, 1]:6,}")
print(f"  False Negatives: {cm_hdbscan[1, 0]:6,}")
print(f"  True Positives:  {cm_hdbscan[1, 1]:6,}")
hdbscan_precision, hdbscan_recall, hdbscan_f1, _ = precision_recall_fscore_support(
    y_true, y_pred_hdbscan, average='binary'
)
print(f"  Precision: {hdbscan_precision:.4f}")
print(f"  Recall: {hdbscan_recall:.4f}")
print(f"  F1-Score: {hdbscan_f1:.4f}")

              precision    recall  f1-score   support

      Normal     0.9991    0.9963    0.9977     25817
     Anomaly     0.4545    0.7692    0.5714       104

    accuracy                         0.9954     25921
   macro avg     0.7268    0.8828    0.7846     25921
weighted avg     0.9969    0.9954    0.9960     25921


Confusion Matrix:
  True Negatives:  25,721
  False Positives:     96
  False Negatives:     24
  True Positives:      80
  Precision: 0.4545
  Recall: 0.7692
  F1-Score: 0.5714


### 10.3 Final Comparison: All 3 Models

In [33]:
print("\n" + "="*60)
print("FINAL MODEL COMPARISON (ALL 3 MODELS)")
print("="*60)

# Create comparison table.
comparison_all = pd.DataFrame({
    'Metric': ['Precision', 'Recall', 'F1-Score', 'True Positives', 'False Negatives', 'False Positives'],
    'Isolation Forest': [
        f"{iso_precision:.4f}",
        f"{iso_recall:.4f}",
        f"{iso_f1:.4f}",
        f"{cm_iso[1, 1]:,}",
        f"{cm_iso[1, 0]:,}",
        f"{cm_iso[0, 1]:,}"
    ],
    'DBSCAN': [
        f"{dbscan_precision:.4f}",
        f"{dbscan_recall:.4f}",
        f"{dbscan_f1:.4f}",
        f"{cm_dbscan[1, 1]:,}",
        f"{cm_dbscan[1, 0]:,}",
        f"{cm_dbscan[0, 1]:,}"
    ],
    'HDBSCAN': [
        f"{hdbscan_precision:.4f}",
        f"{hdbscan_recall:.4f}",
        f"{hdbscan_f1:.4f}",
        f"{cm_hdbscan[1, 1]:,}",
        f"{cm_hdbscan[1, 0]:,}",
        f"{cm_hdbscan[0, 1]:,}"
    ]
})

print("\n" + comparison_all.to_string(index=False))
print(f"\nTotal True Anomalies: {y_true.sum():,}")

# Determine winner.
all_f1_scores = {
    'Isolation Forest': iso_f1,
    'DBSCAN': dbscan_f1,
    'HDBSCAN': hdbscan_f1
}



FINAL MODEL COMPARISON (ALL 3 MODELS)

         Metric Isolation Forest DBSCAN HDBSCAN
      Precision           0.4904 0.8889  0.4545
         Recall           0.4904 0.6154  0.7692
       F1-Score           0.4904 0.7273  0.5714
 True Positives               51     64      80
False Negatives               53     40      24
False Positives               53      8      96

Total True Anomalies: 104


In [34]:
# Visualize confusion matrices for all 3 models.
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Isolation Forest', 'DBSCAN', 'HDBSCAN'),
    specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}, {'type': 'heatmap'}]]
)

fig.add_trace(
    go.Heatmap(
        z=cm_iso,
        x=['Pred Normal', 'Pred Anomaly'],
        y=['True Normal', 'True Anomaly'],
        text=cm_iso,
        texttemplate='%{text}',
        colorscale='Blues',
        showscale=False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Heatmap(
        z=cm_dbscan,
        x=['Pred Normal', 'Pred Anomaly'],
        y=['True Normal', 'True Anomaly'],
        text=cm_dbscan,
        texttemplate='%{text}',
        colorscale='Oranges',
        showscale=False
    ),
    row=1, col=2
)

fig.add_trace(
    go.Heatmap(
        z=cm_hdbscan,
        x=['Pred Normal', 'Pred Anomaly'],
        y=['True Normal', 'True Anomaly'],
        text=cm_hdbscan,
        texttemplate='%{text}',
        colorscale='Greens',
        showscale=False
    ),
    row=1, col=3
)

fig.update_layout(
    title='Confusion Matrices: All 3 Models',
    height=500,
    width=1400
)
fig.show()


# - we see the HDBSCAN has better recall than DBSCABN - but this is happening by more FP. 
# - this we dont want, we want a balance inbwt both - we can consider f-score itself for now.

## 11. Export Models for Production Pipeline

Now export the trained DBSCAN model and all artifacts needed for the production inference pipeline.

In [35]:
import pickle
from pathlib import Path

# Create models directory.
model_dir = Path('../models')
model_dir.mkdir(exist_ok=True)

print("Exporting models and artifacts for production pipeline...")
print("="*60)

# Export DBSCAN model (best performing).
with open(model_dir / 'dbscan_model.pkl', 'wb') as f:
    pickle.dump(dbscan, f)
print(f"✓ DBSCAN model saved to {model_dir / 'dbscan_model.pkl'}")

# Export StandardScaler.
with open(model_dir / 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f"✓ StandardScaler saved to {model_dir / 'scaler.pkl'}")


Exporting models and artifacts for production pipeline...
✓ DBSCAN model saved to ../models/dbscan_model.pkl
✓ StandardScaler saved to ../models/scaler.pkl
